In [1]:
def simple_coro(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

sc = simple_coro(2)
sc

<generator object simple_coro at 0x0000016DD39005F0>

In [2]:
next(sc)

-> Started: a = 2


2

In [3]:
sc.send(1)

-> Received: b = 1


3

In [4]:
try:
    sc.send(3)
except StopIteration:
    print('StopIteration caught')

-> Received: c = 3
StopIteration caught


In [5]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [6]:
avg = averager()
next(avg)

In [7]:
avg.send(10)

10.0

In [8]:
avg.send(20)

15.0

In [9]:
avg.send(30)

20.0

In [10]:
avg.close()

In [11]:
from functools import wraps

def coroutine(func):
    @wraps(func)
    def primer(*args,**kwargs):
        gen = func(*args,**kwargs)
        next(gen)
        return gen
    return primer

In [12]:
@coroutine
def averager_v2():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [13]:
avg = averager_v2()

In [14]:
avg.send(2)

2.0

In [15]:
avg.send(3)

2.5

In [16]:
avg.send(5)

3.3333333333333335

In [17]:
avg.close()

## Returning values
coroutines must terminate normally in order for it to return a value

In [27]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [32]:
avg = averager()
avg.send(1)
avg.send(2)
avg.send(5)

try:
    avg.send(None)
except StopIteration as exc:
    result = exc.value

result

Result(count=3, average=2.6666666666666665)

# Yield From

In [48]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


# the subgenerator
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


# the delegating generator
@coroutine
def grouper(results, key):
    while True:
        results[key] = yield from averager()
        


# the client code, a.k.a. the caller
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        for value in values:
            group.send(value)
        group.send(None)  # important!

    report(results)


# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5}',
              f'averaging {result.average:.2f}{unit}')


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}
main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


# taxi simulation

In [71]:
import random
import collections
import queue

Event = collections.namedtuple('Event', 'time proc action')

def taxi_process(ident, trips, start_time=0):
    """Yield to simulator issuing event at each state change"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')

    yield Event(time, ident, 'going home')
    
def compute_duration(previous_action):
    return randint(3,15)

class Simulator:

    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        """Schedule and display events until time is up"""
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)

        # main loop of the simulation
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break

            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi:', proc_id, proc_id * '   ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))
    
    
DEPARTURE_INTERVAL = 5
NUM_TAXIS = 3
END_TIME = 180

taxis = {i: taxi_process(i, (i + 1) * 2, i * DEPARTURE_INTERVAL)
         for i in range(NUM_TAXIS)}
sim = Simulator(taxis)
sim.run(END_TIME)

taxi: 1     Event(time=0, proc=1, action='leave garage')
taxi: 0  Event(time=10, proc=0, action='leave garage')
taxi: 1     Event(time=12, proc=1, action='pick up passenger')
taxi: 2        Event(time=18, proc=2, action='leave garage')
taxi: 0  Event(time=20, proc=0, action='pick up passenger')
taxi: 1     Event(time=20, proc=1, action='drop off passenger')
taxi: 0  Event(time=28, proc=0, action='drop off passenger')
taxi: 1     Event(time=29, proc=1, action='pick up passenger')
taxi: 1     Event(time=32, proc=1, action='drop off passenger')
taxi: 2        Event(time=33, proc=2, action='pick up passenger')
taxi: 1     Event(time=35, proc=1, action='pick up passenger')
taxi: 0  Event(time=38, proc=0, action='pick up passenger')
taxi: 2        Event(time=38, proc=2, action='drop off passenger')
taxi: 0  Event(time=42, proc=0, action='drop off passenger')
taxi: 1     Event(time=46, proc=1, action='drop off passenger')
taxi: 0  Event(time=53, proc=0, action='going home')
taxi: 2        Eve

In [60]:
help(random.randint)

Help on method randint in module random:

randint(a, b) method of random.Random instance
    Return random integer in range [a, b], including both end points.

